In [109]:
import numpy as np
from main import *
from copy import deepcopy
from random import choice

In [110]:
info = read_file("sample2.dat")
sp = shortest_path(info["graph"])
sp, info["tasks"]

(array([[  0,  32,  46, ..., 271, 226, 261],
        [ 32,   0,  14, ..., 239, 194, 229],
        [ 46,  14,   0, ..., 253, 208, 243],
        ...,
        [271, 239, 253, ...,   0, 119, 154],
        [226, 194, 208, ..., 119,   0,  35],
        [261, 229, 243, ..., 154,  35,   0]]),
 [((0, 1), 32),
  ((1, 0), 32),
  ((1, 2), 14),
  ((2, 1), 14),
  ((1, 3), 17),
  ((3, 1), 17),
  ((3, 4), 56),
  ((4, 3), 56),
  ((8, 9), 20),
  ((9, 8), 20),
  ((10, 11), 32),
  ((11, 10), 32),
  ((11, 15), 29),
  ((15, 11), 29),
  ((12, 15), 13),
  ((15, 12), 13),
  ((12, 13), 7),
  ((13, 12), 7),
  ((14, 16), 26),
  ((16, 14), 26),
  ((14, 17), 38),
  ((17, 14), 38),
  ((17, 18), 41),
  ((18, 17), 41),
  ((18, 19), 32),
  ((19, 18), 32),
  ((18, 20), 38),
  ((20, 18), 38),
  ((20, 21), 17),
  ((21, 20), 17),
  ((21, 74), 24),
  ((74, 21), 24),
  ((22, 30), 40),
  ((30, 22), 40),
  ((30, 31), 58),
  ((31, 30), 58),
  ((31, 32), 30),
  ((32, 31), 30),
  ((31, 33), 43),
  ((33, 31), 43),
  ((31, 34), 86),

In [111]:
depot, capacity = info["depot"], info["capacity"]
num_vehicles = info["num_vehicles"]

In [112]:
def rule1(sp, **kwargs):
    """maximize the distance from the head of task to the depot"""
    return max(kwargs["tasks"], key=lambda x: sp[kwargs["depot"], x[0][0]])

def rule2(sp, **kwargs):
    """minimize the distance from the head of task to the depot"""
    return min(kwargs["tasks"], key=lambda x: sp[kwargs["depot"], x[0][0]])

def rule3(sp, **kwargs):
    """maximize the term dem(t)/sc(t)"""
    return max(kwargs["tasks"], key=lambda x: x[1] / sp[x[0][0], x[0][1]])

def rule4(sp, **kwargs):
    """minimize the term dem(t)/sc(t)"""
    return min(kwargs["tasks"], key=lambda x: x[1] / sp[x[0][0], x[0][1]])

def rule5(sp, **kwargs):
    """use rule1 if the vehicle is less than half full, otherwise use rule2"""
    return rule1(sp, **kwargs) if kwargs["remain_cap"] / kwargs["capacity"] > 0.5 else rule2(sp, **kwargs)

In [113]:
num = 5
tasks = deepcopy(info["tasks"])
dummy_task = ((depot, depot), 0)

In [114]:
def generate_route_by_rule(sp, tasks, depot, capacity, num_vehicles, rule):
    route = [dummy_task]
    
    # 多辆车
    for _ in range(num_vehicles):
        if len(tasks) == 0:
            break

        remain_cap = capacity
        cur_vertex = depot

        while len(tasks) > 0:
            valid_tasks = []
            for task in tasks:
                if task[1] <= remain_cap:
                    valid_tasks.append(task)

            if len(valid_tasks) == 0:
                # 没有有效的任务
                break

            elif len(valid_tasks) == 2:
                # 因为一个tasks有两条记录
                # 只有一个有效任务，最短路径过去
                task = valid_tasks[0] if valid_tasks[0][0][0] < valid_tasks[1][0][0] else valid_tasks[1]

                # 添加路线
                route.append(task)

                # 修改车的位置
                cur_vertex = task[0][0]  

                # 移除task
                tasks.remove(valid_tasks[0])
                tasks.remove(valid_tasks[1])

                # 减少容量
                remain_cap -= task[1]

                # 一定是最后一个了
                break

            else:
                # 有多个任务满足条件，选一个距离最短的
                min_dist = min(
                    valid_tasks, key=lambda x: sp[cur_vertex, x[0][0]])[1]
                # 检查是否有多个距离最短的点
                min_dist_tasks = [t for t in valid_tasks if t[1] == min_dist]
                if len(min_dist_tasks) == 1:  # 1，只有一个
                    task = min_dist_tasks[0]
                else:  # 多个，使用rule
                    task = rule(sp, depot=depot, tasks=tasks, 
                                capacity=capacity, remain_cap=remain_cap)
                
                route.append(task)
                cur_vertex = task[0][0]
                remain_cap -= task[1]
                tasks.remove(task)
                tasks.remove(((task[0][1], task[0][0]), task[1]))
        
        # 一辆车回到终点后，加入dummy_task
        route.append(dummy_task)
    
    return route

In [115]:
def generate_route_random(tasks, capacity, num_vehicles):
    route = [dummy_task]
    
    # 多辆车
    for _ in range(num_vehicles):
        if len(tasks) == 0:
            break

        remain_cap = capacity

        while len(tasks) > 0:
            valid_tasks = []
            for task in tasks:
                if task[1] <= remain_cap:
                    valid_tasks.append(task)

            if len(valid_tasks) == 0:
                # 没有有效的任务
                break

            else:
                task = choice(valid_tasks)
                
                route.append(task)
                remain_cap -= task[1]
                tasks.remove(task)
                tasks.remove(((task[0][1], task[0][0]), task[1]))
        
        # 一辆车回到终点后，加入dummy_task
        route.append(dummy_task)
    
    return route

In [121]:
def generate_route(sp, tasks, depot, capacity, num_vehicles):
    return [generate_route_by_rule(
        sp, deepcopy(info["tasks"]),
        depot, capacity, num_vehicles, rule)
        for rule in [rule1, rule2, rule3, rule4, rule5]] + \
        [generate_route_random(deepcopy(info["tasks"]), depot,
                               capacity, num_vehicles) for _ in range(10)]

routes = generate_route(sp, tasks, depot, capacity, num_vehicles)
len(routes), routes

(15,
 [[((0, 0), 0),
   ((33, 31), 43),
   ((32, 31), 30),
   ((31, 30), 58),
   ((31, 34), 86),
   ((30, 22), 40),
   ((16, 14), 26),
   ((22, 74), 17),
   ((34, 40), 15),
   ((0, 0), 0),
   ((14, 17), 38),
   ((67, 65), 45),
   ((74, 21), 24),
   ((13, 12), 7),
   ((64, 62), 21),
   ((12, 15), 13),
   ((54, 55), 6),
   ((18, 17), 41),
   ((18, 19), 32),
   ((18, 20), 38),
   ((15, 11), 29),
   ((41, 56), 14),
   ((0, 0), 0),
   ((21, 20), 17),
   ((62, 61), 30),
   ((42, 43), 78),
   ((56, 57), 78),
   ((19, 75), 8),
   ((53, 51), 4),
   ((20, 50), 2),
   ((50, 48), 10),
   ((51, 49), 2),
   ((60, 59), 39),
   ((65, 61), 16),
   ((49, 48), 8),
   ((11, 10), 32),
   ((0, 0), 0),
   ((47, 46), 10),
   ((48, 46), 9),
   ((61, 59), 23),
   ((46, 45), 33),
   ((59, 57), 33),
   ((10, 58), 78),
   ((45, 43), 13),
   ((9, 8), 20),
   ((44, 43), 12),
   ((43, 58), 28),
   ((57, 58), 25),
   ((57, 68), 32),
   ((0, 0), 0),
   ((58, 68), 11),
   ((68, 3), 75),
   ((4, 3), 56),
   ((3, 1), 17),